# Generative model for MOT images
## Team: Loic Anderegg, Daniel Ang, Andrei Gheorghe

The high level objective of this project is to get the temperature of a gas of molecules or atoms trapped in a magneto-optical trap (MOT). For this, the first goal is to fit images of the MOT taken on a CCD camera to 2D Gaussians, which is our basic guess for its shape. On top of shape we also model multiple sources of uncertainty, from statistical uncertainty in our 2D Gaussian guess to uncertainties arising from shot noise, readout noise or scattered light noise. 

To actually get the temeprature of the gas we physically have to release the molecules/atoms from the trap and let them expand. From the expansion rate we can then infer the temeprature. Thus, the second goal is to analyze multiple such images taken at different times after the release from the trap. For each moment in time we use our model to fit parameters such as $\sigma_{\mathrm{x}}$ and $\sigma_{\mathrm{y}}$, i.e. the widths of the MOT, which we then use to obtain the temperature.

Ideally the package should be felxible enough to be used on any MOT image. Members from our group have data for MOTs made from molecules of CaF, or from atoms of Rb or K. We build our package by first testing it on the molecular MOT, since that is the noisiest one and it also lends itself to expansions of the model such as considering the slosh of particles in the trap itself, which is not negligible in this case (in other words all trapped molecules have a non negligible initial velocity in the same direction).

In [1]:
%matplotlib inline 

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import math

import pandas as pd
import seaborn as sns

import scipy.integrate as integrate
import scipy.special as sp
import scipy.optimize as spop

import emcee

### Bare model

The model is represented by a 2D Gaussian, with random peak amplitude, width and peak position.

We thus start by defining the 2D and 1D Gaussian function:

In [2]:
def gaussian_2d(x, y, center_x, center_y, amplitude, sigma_x, sigma_y):
    return amplitude*np.exp(-(x-center_x)/(2*sigma_x**2))*np.exp(-(y-center_y)/(2*sigma_y**2))

def gaussian_1d(z, center_z, sigma_z, amplitude):
    return amplitude*np.exp(-(z-center_z)**2/(2*sigma_z**2))

We next define the model:

In [3]:
def model(x, y, theta):
    center_x, center_y, amplitude, sigma_x, sigma_y, background_offset = theta
    return gaussian_2d(x, y, center_x, center_y, amplitude, sigma_x, sigma_y) + background_offset

For an example of how a generic image generated with this model looks like, look at model_MOTonly.ipynb file.

### Uncertainties

We consider three possible uncertainty sources:

- a Gaussian statistical uncertainty in the model, $\epsilon_{\mathrm{i}}$;
- a Poissonian uncertainty, given by shot noise in the CCD camera;
- a Gaussian uncertainty, given by scattered light;
- a Gaussian uncertainty with a prior that is proportionally higher if the pixels below it are brighter.

## Likelihood function

To construct the likelihood function we need to consider all the uncertainties. Thus, each data point is given by: 

$$y_{\mathrm{i}} = z_{\mathrm{i}} + e_{\mathrm{i}} = m(x_{\mathrm{i}}, \theta) + \epsilon_{\mathrm{i}} + f_{\mathrm{i}} + g_{\mathrm{i}} + sct_{\mathrm{i}},$$

where $m(x_{\mathrm{i}}, \theta)$ is the model data and $\theta$ the set of parameters that go into the data.

Following the methods of Gregory 4.8, we have the probability distribution for the proposition $Z_i$ that the model we considered gives for the $i$-th data point values in the range $z_{\mathrm{i}}$ to $z_{\mathrm{i}} + dz_{\mathrm{i}}$:

$$p(Z_{\mathrm{i}}|M, \theta, I) = f_{\mathrm{Z}}(z_{\mathrm{i}}) = \frac{1}{\sqrt{2 \pi}\sigma_{\mathrm{mi}}} exp \left( \frac{- \epsilon_i}{2 \sigma_{\mathrm{mi}}} \right)$$.

Note that for simplicity we will consider $\epsilon_{\mathrm{i}} = \epsilon$ and $\sigma_{\mathrm{mi}} = \sigma_{\mathrm{m}}$, for all $i$.

Next the shot noise component is given by:

$$p(F_{\mathrm{i}}|M, \theta, I)  = e^{-\mu_fi} \frac{\mu_{\mathrm{fi}}^n}{n!}.$$

A similar expression applies to the scattered light noise, with expected photon number $\mu_{\mathrm{scti}}$.

Finally, include the readout noise. For now, only consider it as a simple Gaussian but can easily add a prior of the form sum(z(x0, y-i)), for $i$ from $0$ to $y0$, where (x0, y0) are the coordiantes of the pixel we are analyzing. Then the noise is really this Gaussian likelihood times the prior.

$$p(G_{\mathrm{i}}|M, \theta, I) = \frac{1}{\sqrt{2 \pi}\sigma_{\mathrm{gi}}} exp \left( \frac{- g_i}{2 \sigma_{\mathrm{gi}}} \right)$$

The likelihood for each pixel is then the convolution of the noise sources and the total likelihood is the product of all likelihoods for each pixel (so for log of the likelihood we perform a sum). For the first stages of the project we perform the convolution directly:

NOTE: In this simple version of the model, the two Gaussian uncertainties convolve to a Gaussian with $\sigma = \sigma_{\mathrm{m}} + \sigma_{\mathrm{g}}$

In [14]:
def log_likelihood(x, y, z, sigma_m, sigma_g, mu_f, mu_sct, theta):
    '''
    theta: model parameters
    x, y: independent data (arrays of size 250)
    z: measurement (brightness of pixel) 
    sigma_m: uncertainty in the model chosen
    sigma_g: uncertainty from CCD camera readout noise
    mu_f: average number of photons due to shot noise
    mu_sct = average number of photons due to scattered light
    '''
    center_x, center_y, amplitude, sigma_x, sigma_y, background_offset = theta
    
    sigma = sigma_m + sigma_g
    prob_gaussian = gaussian_1d(z, model(x, y, theta), sigma, 1) 
    prob_poisson = np.random.poisson(mu_f) + np.random.poisson(mu_sct)
    
    log_likelihood = np.sum(np.log(prob_gaussian + prob_poisson)) 
    
    # Should we just add the Poisson shot noise on top of whatever the model + model uncertainty gives?